# Декораторы

# Задача 1: Суммы прогрессий

Необходимо реализовать функционал для подсчета суммы первых n + 1 - членов арифметической и геометрической прогрессии с возможностями настройки шага и значения первого члена. n соответствует числу вызовов функции по подсчету суммы. 

Предполагаемые сценарии использования:

```python

sum_arithmetic = make_arithmetic_progression_sum(first_member=2, step=0.5)
sum_geometric = make_geometric_progression_sum(first_member=1, step=0.5)

print(sum_arithmetic())
print(sum_arithmetic())
print('')
print(sum_geometric())
print(sum_geometric())
```

*Вывод*:
```console
4.5
7.5

1.5
1.75
```

In [2]:
def make_arithmetic_progression_sum(first_member: float, step: float):
    calls_amount = 0
    current_sum = first_member

    def arithmetic_progression_sum() -> float:
        nonlocal calls_amount, current_sum

        calls_amount += 1
        current_sum += step * calls_amount + first_member
        return current_sum

    return arithmetic_progression_sum


def make_geometric_progression_sum(first_member: float, step: float):
    last_member = first_member
    current_sum = first_member

    def geometric_progression_sum() -> float:
        nonlocal last_member, current_sum

        last_member *= step
        current_sum += last_member
        return current_sum

    return geometric_progression_sum

sum_arithmetic = make_arithmetic_progression_sum(first_member=2, step=0.5)
sum_geometric = make_geometric_progression_sum(first_member=1, step=0.5)

print(sum_arithmetic())
print(sum_arithmetic())
print('')
print(sum_geometric())
print(sum_geometric())

4.5
7.5

1.5
1.75


# Задача 2: Среднее

Предположим, что мы занимаемся инвестициями и у нас есть некоторый портфель акций. Каждый день наш портфель приносит нам некоторый доход или убыток. Мы задались целью: каждый день фиксированного периода определять средний доход (или убыток), который мы получаем. С этой целью мы реализовали функцию get_avg(), принимающую на вход значение заработка на сегодняшний день. Наша функция вычисляет среднее в течении определнного фиксированного периода, скажем, 30 дней, после чего обнуляется и начинает вычислять среднее заново, но уже по новым значениям. 

Также у нас есть друзья инвесторы, которые оценили разработанный нами функционал и хотели бы заполучить свой экземпляр функции get_avg, для подсчета своего дохода в течении интересующего их промежутка времени.

Ваша задача: реализовать функционал, для получения произвольного числа независимых функций get_avg(). В момент создания функции сообщается длительность периода расчета среднего, по достижении которого среднее начинает расчитываться заново, а также наш начальный доход. При каждом вызове функции передается число - заработок в текущий день.

Предполагаемые сценарии использования:

```python

get_avg1 = make_averager(accumulation_period=2)
print(get_avg1(78))
print(get_avg1(-17))
print(get_avg1(52))
```

*Вывод*:
```console
78.0
30.5
52.0
```

In [3]:
def make_averager(accumulation_period: int):
    accumulation_period = int(accumulation_period)

    if accumulation_period < 1:
        raise ValueError("unexpected accumulative period")
    
    sum_current = 0
    day_current = 0

    def count_avg_sum(number: float):
        nonlocal sum_current, day_current
        
        day_current += 1
        number = float(number)

        if day_current > accumulation_period:
            day_current = 1
            sum_current = 0

        sum_current += number

        return sum_current / day_current

    return count_avg_sum

get_avg1 = make_averager(accumulation_period=2)
print(get_avg1(78))
print(get_avg1(-17))
print(get_avg1(52))

78.0
30.5
52.0


# Задача 3: Сбор статистик

Предположим, что мы работаем в отделе аналитики некоторой компании. В компании также существуют другие отделы, которые разрабатывают некоторые функции для осуществления сложных вычислений. Также в нашей компании существует отдел планирования, который следит за исполнением сроков реализации той или иной функции, и в случае, если разработка затягивается, начинает торопить разработчиков. В таком случае разработчики пишут медленный код на скорую руку, что расстраивает заказчиков.

Наша задача, как аналитиков, собрать статистику по проблемным функциям. Нас интересует количество вызовов функции, а также среднее время выполнения функции. Все статистики собираются в отдельную базу данных - специальный единый словарь. Более того, статистика должна собираться не для всех функций, а только для функций, зарегестрированных в базе данных. Затем эта информация будет передана начальству, чтобы в скорейшее время заняться переписанием долгих и популярных функций.

Ваша задача реализовать функционал для регистрации функций в БД и сбора статистик. 

In [3]:
from functools import wraps
import time

functions_register = {}

def func_register(func):
    def wrapper(*args, **kwargs):
        if func not in functions_register:
            functions_register[func] = [0, 0]
        
        return func

    return  wrapper


def get_statistics(func):
    if func not in functions_register:
            raise RuntimeError(
                f"{func.__name__} is not registered\
                can not get statistics"
               )
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        time_start = time.time()
        func(*args, **kwargs)
        time_end = time.time()

        func_register[func][0] += time_end - time_start
        func_register[func][0] += 1
        return func
    
    return wrapper

# Задача 4: наивный LRU-кэш

Представим, что вы являетесь сотрудником департамента оптимизаций некоторой компании, занимающейся разработкой ПО для научных вычислений. Раздел аналитиков предоставил вам исследование, согласно которому значительная часть функций в вашей компании работает очень медленно и должна быть оптимизирована. Более того, согласно исследованию, вызовы этих функций в основном осуществляются с ограниченным множеством аргументов. Для оптимизации этих функций вы решили использовать LRU-кэш:

- вы заранее фиксируете размер кэша - памяти, выделенной для хранения результатов вычислений функции;  
- в кэше хранится следующая информация: аргументы вызова - результат;  
- помимо этого для каждой пары хранится время их последнего вызова;  
- в случае достижения объема кэша установленной границы удаляются значения, чьи времена последних вызовов являются самыми старыми;  

Для применения данного подхода ко всем проблемным функциям, не переписывая сами функции, вы решили реализовать параметрический декоратор, т.к. для разных функций требуется разный размер кеша.

Ваша задача: реализовать описанный декоратор.

In [23]:
import time, sys


def lru_cahce(args_amount):

    def func_func(func):

        def wrapper(*args, **kwargs):
            nonlocal args_amount
            
            founded_results = {}
            min_last_time = time.time() + 9999999999999
            result_last_time = None

            for arg in args:
                last_used_time = time.time()
                func_result = func(arg)

                if func_result in founded_results:
                    pass
                elif len(founded_results) < args_amount:
                    founded_results[func_result] = (arg, last_used_time)
                else:
                    for result in founded_results:
                        if result[1] < min_last_time:
                            min_last_time = result[1]
                            result_last_time = result
                    founded_results[result_last_time] = (arg, last_used_time)

            return 
        
        return wrapper
    
    return func_func

sys.setrecursionlimit(1000000000)

@lru_cahce(args_amount=100)
def fib(number):
    if number < 2:
        return 1
    return fib(number - 2) + fib(number - 1)

print(fib(100))

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'